1. Попробуйте обучить три сети представленные в тетради.
(Разморозкой слоев заниматься не надо, разберем разморозку на уроке.)
Сохраните модели.

2. Напишите функцию принимающую изображение и дающую прогноз (кот /собака) с использованием всех трех моделей.

##**1) Обучение трех сетей представленных в тетради.**

In [ ]:
from google.colab import drive
from google.colab import files

In [ ]:
!mkdir GD

In [ ]:
!mkdir GD/'My Drive'/

In [ ]:
!ls /GD/'My Drive'/

ls: cannot access '/GD/My Drive/': No such file or directory


In [ ]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


In [ ]:
!cp -r /GD/'My Drive'/PetImages.zip .

In [ ]:
!ls 

GD  PetImages.zip  sample_data


In [ ]:
!unzip PetImages.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
!ls 

GD  PetImages  PetImages.zip  sample_data


In [ ]:
!mkdir PetImages/validation_data

In [ ]:
#!mv PetImages/Cat/ PetImages/train/Cat/

In [ ]:
#!mv PetImages/Dog/ PetImages/train/Dog/

In [ ]:
#!mkdir PetImages/test

In [ ]:
#!mkdir PetImages/test/Cat

In [ ]:
!mkdir PetImages/validation_data/Dog

In [ ]:
!mkdir PetImages/validation_data/Cat

In [ ]:
!ls PetImages/validation_data

Cat  Dog


## разделение данных на тестовое и обучающее множество

In [ ]:
import os, shutil
import random as rn

In [ ]:
train_dirs = ['PetImages/train/Dog', 
              'PetImages/train/Cat']

test_dirs =  ['PetImages/validation_data/Dog', 
              'PetImages/validation_data/Cat']

In [ ]:
# Страшный код

for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames)//10

    filesToCopy = []
    for j in range(toCopy):
        filesToCopy.append(rn.choice(fnames))
    
    # cOPY
         
    for fi in filesToCopy:
        src = os.path.join(train_dirs[i],fi)
        dst = os.path.join(test_dirs[i], fi)
        shutil.copyfile(src,dst)

    
    for fi in filesToCopy:
        try:
            src = os.path.join(train_dirs[i],fi)
            os.remove(src)
            print(src+' deleted')
            #time.sleep(1)
        except:
            pass


In [ ]:
# осмотр получившихся директорий

In [ ]:
!ls PetImages/validation_data/Dog

## подготовка генераторов

In [ ]:
# Воспоминание о генераторах

In [ ]:
def int_generator ():
    i = 0
    while True:
        yield i
        i += 1
        

In [ ]:
j = 0
for item in int_generator():
    print (item)
    j += 1
    if j > 10:
        break

In [ ]:
# взрослый генератор

In [ ]:
Pic_Y, Pic_X = 180, 180
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18502 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
# Обучение модели с 0

model_1 = models.Sequential()

model_1.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model_1.add(layers.MaxPooling2D(pool_size=2))

model_1.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model_1.add(layers.MaxPooling2D(pool_size=2))

model_1.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model_1.add(layers.MaxPooling2D(pool_size=2))

model_1.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model_1.add(layers.MaxPooling2D(pool_size=2))
model_1.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model_1.add(layers.Flatten())
model_1.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 128)      0

In [ ]:
model_1.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_accuracy")]

In [ ]:
history = model_1.fit(train_generator, epochs=3, validation_data=test_generator,
    callbacks=callbacks)

Epoch 3/3
290/290 [==============================] - 83s 287ms/step - loss: 0.4464 - accuracy: 0.7896 - val_loss: 0.3760 - val_accuracy: 0.8362


In [ ]:
!ls

convnet_from_scratch.h5  GD  PetImages	PetImages.zip  sample_data


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp convnet_from_scratch.h5 /GD/'My Drive'/convnet_from_scratch.h5

## Добавляем расширение данных 

In [ ]:
model_2 = models.Sequential()

model_2.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))
model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model_2.add(layers.Flatten())
model_2.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model_2.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18502 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
history = model_2.fit(train_generator, epochs=5, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/5
290/290 [==============================] - ETA: 0s - loss: 0.6767 - accuracy: 0.5710

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 208s 711ms/step - loss: 0.6767 - accuracy: 0.5710 - val_loss: 0.6167 - val_accuracy: 0.6466
Epoch 2/5
290/290 [==============================] - 203s 701ms/step - loss: 0.5862 - accuracy: 0.6883 - val_loss: 0.5046 - val_accuracy: 0.7503
Epoch 3/5
290/290 [==============================] - 199s 685ms/step - loss: 0.5140 - accuracy: 0.7487 - val_loss: 0.4599 - val_accuracy: 0.7862
Epoch 4/5
290/290 [==============================] - 200s 689ms/step - loss: 0.4644 - accuracy: 0.7852 - val_loss: 0.4254 - val_accuracy: 0.8104
Epoch 5/5
290/290 [==============================] - 199s 686ms/step - loss: 0.4237 - accuracy: 0.8044 - val_loss: 0.3398 - val_accuracy: 0.8496


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp augmentation_model.h5 /GD/'My Drive'/augmentation_model.h5

## Перенос знаний!

In [ ]:
# закгрузка сверточных слоев VGG16

conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Заморозка сверточных слоев
conv_base.trainable = False

In [ ]:
# Конструируем модель
model_3 = models.Sequential()
model_3.add(conv_base)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_3.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="VGG_FIX_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
# фитим на тех же генераторах то есть с расширением данных
history = model_3.fit(train_generator, epochs=1, validation_data=test_generator, 
    callbacks=callbacks)

290/290 [==============================] - ETA: 0s - loss: 0.3817 - accuracy: 0.8233

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 242s 814ms/step - loss: 0.3817 - accuracy: 0.8233 - val_loss: 0.2491 - val_accuracy: 0.8926


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp VGG_FIX_model.h5 /GD/'My Drive'/VGG_FIX_model.h5

In [ ]:
## Fine Turning

In [ ]:
# # Размораживаем 4 верхних свертки
# conv_base.trainable = True
# for layer in conv_base.layers[:-4]:
#     layer.trainable = False

In [ ]:
# model_3.compile(loss="binary_crossentropy",
#               optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
#               metrics=["accuracy"])

# callbacks = [tf.keras.callbacks.ModelCheckpoint(
#         filepath="fine_tuning.h5",
#         save_best_only=True,
#         monitor="val_loss")]


In [ ]:
# history = model_3.fit(train_generator, epochs=1, validation_data=test_generator, 
#     callbacks=callbacks)

##**2) Написание функции принимающей изображение и дающую прогноз (кот /собака) с использованием всех трех моделей.**

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
import os
import numpy as np

Загрузка моделей с диска

In [ ]:
!cp /GD/'My Drive'/convnet_from_scratch.h5 .

In [ ]:
!cp /GD/'My Drive'/augmentation_model.h5 .

In [ ]:
!cp /GD/'My Drive'/VGG_FIX_model.h5 .

In [ ]:
model1 = tf.keras.models.load_model("convnet_from_scratch.h5")
model2 = tf.keras.models.load_model("augmentation_model.h5")
model3 = tf.keras.models.load_model("VGG_FIX_model.h5")

Функция принимающая изображение и дающая прогноз (кот /собака) с использованием всех трех моделей

In [ ]:
def prediction(image, model1, model2, model3):
    # Cжимаем изображение
    image = image.resize((180, 180))
    image_array = np.array(image) / 255.0

    # Добавляем дополнительное измерение для соответствия размеру входных данных модели
    image_input = np.expand_dims(image_array, axis=0)

    # Получаем предсказания от каждой модели
    pred1 = model1.predict(image_input)
    pred2 = model2.predict(image_input)
    pred3 = model3.predict(image_input)

    # Усредняем предсказания
    avg_pred = (pred1 + pred2 + pred3) / 3.0

    # Определяем кошка или собака
    if avg_pred >= 0.5:
        return "Dog"
    else:
        return "Cat"


In [ ]:
import random

Случайный выбор изображения из двух папок

In [ ]:
i=0
pa1 = "PetImages/validation_data/Dog"
pa2 = "PetImages/validation_data/Cat"
path1 = Path(pa1).rglob("*.jpg")
path2 = Path(pa2).rglob("*.jpg")
path = random.choice((path1, path2))

Выполнение функции, получение пргноза

In [ ]:
try:
  img = PIL.Image.open(next(path))
  result = prediction(img, model1, model2, model3)
  print(f"На изображении {next(path)} - {result}.")
except PIL.UnidentifiedImageError:
  i+=1
  print(next(path), i)
  os.remove(next(path))

1/1 [==============================] - 0s 21ms/step
На изображении PetImages/validation_data/Cat/10560.jpg - Cat.


In [ ]:
# ## Оценка модели на проверочном множестве.
# model = tf.keras.models.load_model("convnet_from_scratch.h5")
# test_loss, test_acc = model.evaluate(train_generator)
# print(f"Test accuracy: {test_acc:.3f}")

290/290 [==============================] - 179s 617ms/step - loss: 0.4396 - accuracy: 0.8003
Test accuracy: 0.800


In [ ]:
# import PIL
# from pathlib import Path
# from PIL import UnidentifiedImageError
# import os
# i=0
# pa = "PetImages/train/Cat"
# path = Path(pa).rglob("*.jpg")
# for img_p in path:
#     try:
#         img = PIL.Image.open(img_p)
#     except PIL.UnidentifiedImageError:
#             i+=1
#             print(img_p, i)
#             os.remove(img_p)

PetImages/train/Cat/666.jpg 1


In [ ]:
#PetImages/train/Dog/11702.jpg 1